# [ParslFest 2025](https://parsl-project.org/parslfest/parslfest2025.html)

# [Accelerating QMCpy Notebook Tests with Parsl](https://www.figma.com/slides/k7EUosssNluMihkYTLuh1F/Parsl-Testbook-Speedup?node-id=1-37&t=WnKcu2QYO8JXvtpP-0)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QMCSoftware/QMCSoftware/blob/develop/demos/talk_paper_demos/parsel_fest_2025/parsl_fest_2025.ipynb)

Joshua Herman, Brandon Sharp, and Sou-Cheng Choi, QMCPy Developers

Aug 28 -- 29, 2025

Updated: Dec 1, 2025


**Requirements**:

* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

In [1]:
try:
    import parsl as pl
except ModuleNotFoundError:
    !pip install -q parsl

In [2]:
import sys
import os
import time
import parsl as pl

# Ensure the path to the booktests directory is included (robust finder)
def _find_repo_root(start=os.getcwd()):
    cur = start
    while True:
        if os.path.exists(os.path.join(cur, 'pyproject.toml')):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            raise FileNotFoundError('repo root not found')
        cur = parent

sys.path.append(os.path.join(_find_repo_root(), 'test', 'booktests'))

# Configuration flags
force_compute = True
is_debug = False

# Create output directory if it doesn't exist
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

## 2. Parsl

1. Install Parcel
2. Run the tests in parallel with Parsl

### 2.2 Create a Parsl Test Runner

In [3]:
import parsl_test_runner
import inspect

# See only functions
print("Functions:")
functions = inspect.getmembers(parsl_test_runner, inspect.isfunction)
for name, func in functions:
    print(f"- {name}")
print("\n" + "="*50)

# Get help on specific function
print("Help for execute_parallel_tests:")
help(parsl_test_runner.execute_parallel_tests)

Functions:
- _kill_interchange_processes
- bash_app
- execute_parallel_tests
- generate_summary_report
- get_runtime
- main
- optimal_schedule
- print_schedule
- reload_parsl_config
- sort_by_runtime

Help for execute_parallel_tests:
Help on function execute_parallel_tests in module parsl_test_runner:

execute_parallel_tests()
    Execute all tb_*.py booktests in parallel using Parsl.



### 2.3 Run the Notebooks in Parallel with Parsl

In [ ]:
import os
import time
import uuid

# Configuration flags
force_compute = True   # re-run even if CSV exists
is_debug = False       # if True, only run a small subset of tests

# Ensure output directory exists
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Read sequential baseline time (from 01_sequential.ipynb)
seq_fname = os.path.join(output_dir, "sequential_time.csv")
with open(seq_fname, "r") as f:
    sequential_time = float(f.read().strip())
print(f"Sequential baseline time: {sequential_time:.2f} s")

for workers in [1, 2, 4, 8]:
    print("\n" + "=" * 60)
    print(f"=== RUNNING PARSL BOOKTESTS WITH {workers} WORKER(S) ===")

    # Pass desired worker count to parsl_test_runner via environment
    os.environ["QMC_PARSL_WORKERS"] = str(workers)

    # Output CSV for this worker setting
    par_fname = os.path.join(output_dir, f"parallel_times_{workers}.csv")

    if not os.path.exists(par_fname) or force_compute:
        execution_id = str(uuid.uuid4())[:8]
        print(f"Execution ID: {execution_id}")
        print(f"Starting parallel test execution with {workers} workers...")

        start_time = time.time()
        if is_debug:
            # Debug mode: restrict to a couple of tests
            !(cd ../../.. && make booktests_parallel_no_docker TESTS="tb_quickstart tb_qmcpy_intro")
        else:
            # Full booktest run in parallel
            !(cd ../../.. && make booktests_parallel_no_docker)
        parallel_time = time.time() - start_time

        # Compute speedup using sequential baseline
        speedup = sequential_time / parallel_time

        print(f"\n=== RESULTS FOR EXECUTION {execution_id} (workers={workers}) ===")
        print(f"Sequential time:  {sequential_time:.2f} s")
        print(f"Parallel time:    {parallel_time:.2f} s")
        print(f"Speedup by Parsl: {speedup:.2f}x")
        print(f"=== END EXECUTION {execution_id} ===")

        # Save this run to CSV
        with open(par_fname, "w") as f:
            f.write("workers,times,speedup\n")
            f.write(f"{workers},{parallel_time:.2f},{speedup:.2f}\n")

        print(f"Saved results to {par_fname}")
    else:
        print(f"Skipping run for {workers} workers (file already exists: {par_fname})")

print("\nAll Parsl runs completed. You can now run 03_visualize_speedup.ipynb.")


Sequential baseline time: 768.54 s

=== RUNNING PARSL BOOKTESTS WITH 1 WORKER(S) ===
Execution ID: c50fe4e8
Starting parallel test execution with 1 workers...
Checking notebook ↔ booktest coverage...
    Skipping demos/DAKOTA_Genz/dakota_genz.ipynb (requires large manual file / heavy memory use)


    Skipping demos/talk_paper_demos/Argonne_Talk_2023_May/Argonne_2023_Talk_Figures.ipynb (heavy LaTeX + many figures; skipped in booktests_no_docker)
    Skipping demos/talk_paper_demos/MCQMC2022_Article_Figures/MCQMC2022_Article_Figures.ipynb (MCQMC 2022 article figures; not run in CI)
    Skipping demos/talk_paper_demos/ProbFailureSorokinRao/prob_failure_gp_ci.ipynb (prob_failure_gp_ci talk demo; heavy GP / prob. failure example)
    Skipping demos/talk_paper_demos/Purdue_Talk_2023_March/Purdue_Talk_Figures.ipynb (Purdue 2023 talk figures; not a CI booktest target)
    Skipping demos/talk_paper_demos/parsel_fest_2025/01_sequential.ipynb (helper notebook for parsl_fest_2025; not a standalone booktest)
    Skipping demos/talk_paper_demos/parsel_fest_2025/02_parallel.ipynb (helper notebook for parsl_fest_2025; not a standalone booktest)
    Skipping demos/talk_paper_demos/parsel_fest_2025/03_visualize_speedup.ipynb (helper notebook for parsl_fest_2025; not a standalone booktest)
    Sk

In [7]:
!date
!ls -ltr output

Mon Dec  1 18:00:44 UTC 2025
total 212
-rw-rw-rw- 1 codespace codespace      7 Dec  1 16:52 sequential_time.csv
-rw-rw-rw- 1 codespace codespace     36 Dec  1 17:40 parallel_times_2.csv
-rw-rw-rw- 1 codespace codespace     36 Dec  1 17:45 parallel_times_4.csv
-rw-rw-rw- 1 codespace codespace     36 Dec  1 17:52 parallel_times_6.csv
-rw-rw-rw- 1 codespace codespace     36 Dec  1 17:57 parallel_times_8.csv
-rw-rw-rw- 1 codespace codespace 191903 Dec  1 17:59 parallel_times_speedup.png
-rw-rw-rw- 1 codespace codespace     36 Dec  1 18:00 parallel_times_1.csv


In [8]:
import platform

if platform.system().lower() == 'linux':
    !uname -a
    !nproc --all
    !awk '/MemTotal/ {printf "%.2f GB\n", $2/1024/1024}' /proc/meminfo

Linux codespaces-6c6933 6.8.0-1030-azure #35~22.04.1-Ubuntu SMP Mon May 26 18:08:30 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
16
62.80 GB
